In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from itertools import combinations
from random import shuffle

In [2]:
data = pd.read_excel("Fobal.xlsx")
num_jugadores=data.Jugador.dropna().nunique()-2
data_jugadores=data[:num_jugadores]
data_resultados=data[data.Jugador.isin(["Equipo 0","Equipo 1"])]

In [3]:
data_jugadores

,Jugador,2022-12-29 00:00:00,2023-01-05 00:00:00,2023-01-12 00:00:00,2023-01-19 00:00:00,2023-01-26 00:00:00,2023-02-02 00:00:00,2023-02-09 00:00:00,2023-02-16 00:00:00
0,PabloB,NaN,NaN,NaN,0.0,1.0,NaN,0.0,1.0
1,Santi,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN
2,Nuss,NaN,0.0,1.0,0.0,0.0,1.0,0.0,NaN
3,Ale,1.0,0.0,1.0,0.0,1.0,1.0,NaN,1.0
4,Delpe,NaN,NaN,1.0,0.0,1.0,1.0,0.0,NaN
5,German (Ivo),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
6,Negri,NaN,NaN,0.0,1.0,0.0,NaN,1.0,0.0
7,Chehebar,1.0,1.0,NaN,1.0,0.0,1.0,0.0,0.0
8,Ivo,0.0,1.0,NaN,1.0,1.0,0.0,1.0,0.0
9,Charly,1.0,0.0,0.0,1.0,NaN,NaN,NaN,1.0


In [4]:
data_resultados

,Jugador,2022-12-29 00:00:00,2023-01-05 00:00:00,2023-01-12 00:00:00,2023-01-19 00:00:00,2023-01-26 00:00:00,2023-02-02 00:00:00,2023-02-09 00:00:00,2023-02-16 00:00:00
31,Equipo 0,9.0,10.0,9.0,7.0,10.0,7.0,9.0,8.0
32,Equipo 1,9.0,8.0,7.0,12.0,10.0,10.0,6.0,13.0


In [5]:
X=[]
y=[]
for date in data.columns[1:]:
    eq0=[1 if i==0 else 0 for i in data_jugadores[date]]
    eq1=[1 if i==1 else 0 for i in data_jugadores[date]]
    X.append(eq0+[-i for i in eq1])
    X.append(eq1+[-i for i in eq0])
    y.append(data_resultados[data_resultados.Jugador=="Equipo 0"][date].iloc[0])
    y.append(data_resultados[data_resultados.Jugador=="Equipo 1"][date].iloc[0])
    

In [6]:
reg = linear_model.Ridge(alpha=10)
reg.fit(X, y)

Ridge(alpha=10)

In [7]:
def norm(l):
    mn=min(l)
    mx=max(l)
    return [round((i-mn)/(mx-mn)*40+60,1) for i in l]

In [8]:
ataq=reg.coef_[:num_jugadores]
ataq=norm(ataq)
defe=reg.coef_[num_jugadores:]
defe=norm(defe)
data_jugadores["Ataque"]=ataq
data_jugadores["Defensa"]=defe

C:\Users\pablo\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [9]:
data_jugadores["Participaciones"] = np.count_nonzero(data_jugadores==0, axis=1)+np.count_nonzero(data_jugadores==1, axis=1)
    

In [10]:
scores=data_jugadores[["Jugador","Ataque","Defensa","Participaciones"]].sort_values(by=["Participaciones","Ataque","Defensa"], ascending=False)
scores.style.format({'Ataque': "{}",'Defensa': "{}"}).hide_index()

Jugador,Ataque,Defensa,Participaciones
Chehebar,88.8,82.6,7
Ale,84.4,69.4,7
Ivo,76.4,64.6,7
Nuss,73.1,83.5,6
Charly,100.0,100.0,5
Negri,80.2,76.5,5
Delpe,71.3,81.5,5
Maxi,60.0,66.0,5
PabloB,86.9,76.2,4
Piombo,85.4,78.4,3


In [11]:
def Pronostico(nombres0,nombres1):
    eq0=[1 if j in nombres0 else 0 for j in data_jugadores["Jugador"]]
    eq1=[1 if j in nombres1 else 0 for j in data_jugadores["Jugador"]]
    res=reg.predict([eq0+[-i for i in eq1],eq1+[-i for i in eq0]])
    return res

In [12]:
nombres0={"Ale", "Maxi", "Charly", "PabloB", "Negri"}
nombres1={"Piombo", "Kordon", "Ivo", "Delpe", "Santi"}
for nombre in nombres0 | nombres1:
    if nombre not in data_jugadores["Jugador"].values:
        print(nombre+" no esta en la tabla.")
res=Pronostico(nombres0,nombres1)
print("Pronostico:")
print(*nombres0, sep = ", ",end = '')
print(": "+str(round(res[0],1))) 
print(*nombres1, sep = ", ",end = '')
print(": "+str(round(res[1],1))) 

Pronostico:
Maxi, Charly, PabloB, Ale, Negri: 9.5
Santi, Piombo, Delpe, Ivo, Kordon: 8.7


In [13]:
def ArmarEquipos(nombres):
    best=float('inf')
    for nombres0 in combinations(nombres[:-1], 5):
        nombres1=[n for n in nombres if n not in nombres0]
        res=Pronostico(nombres0,nombres1)
        if abs(res[0]-res[1])<best:
            best=abs(res[0]-res[1])
            best_nombres0=nombres0
            best_nombres1=nombres1
    return best_nombres0, best_nombres1

In [14]:
nombres=["Charly", "Negri", "Maxi", "Ale", "Ivo", "Kordon", "Piombo", "Santi", "PabloB","Delpe"]
for nombre in nombres:
    if nombre not in data_jugadores["Jugador"].values:
        print(nombre+" no esta en la tabla.")
shuffle(nombres)
nombres0, nombres1=ArmarEquipos(nombres)
res=Pronostico(nombres0,nombres1)
print("Equipos:")
print(*nombres0, sep = ", ") 
print(*nombres1, sep = ", ") 

Equipos:
PabloB, Ivo, Piombo, Delpe, Negri
Ale, Kordon, Santi, Charly, Maxi
